<a href="https://colab.research.google.com/github/masterPLIA2425/ta-practica1-manapole/blob/main/ejercicio_obligatorio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ejercicios

Esta sesión consta de un ejercicio mínimo y de varias ampliaciones que no son obligatorias.

## Ejercicio mínimo (5 puntos)

Usando el notebook de explicación construye un modelo para el dataset de neutralización y súbelo a HuggingFace. Incluye en la siguiente celda el enlace a tu modelo en HuggingFace y guarda los cambios en GitHub.  


Enlace al modelo: https://huggingface.co/manapole/mbart-neutralization

In [1]:
!pip install datasets evaluate transformers[sentencepiece] sacrebleu accelerate -Uq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 342.1/342.1 kB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 43.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 52.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 37.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from huggingface_hub import notebook_login

notebook_login()

In [3]:
from datasets import load_dataset

In [4]:
dataset = load_dataset("hackathon-pln-es/neutral-es")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/4.38k [00:00<?, ?B/s]

dataset_infos.json:   0%|          | 0.00/721 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/177k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/3.71k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/3513 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/96 [00:00<?, ? examples/s]

In [5]:
dataset

DatasetDict({
    train: Dataset({
        features: ['gendered', 'neutral'],
        num_rows: 3513
    })
    test: Dataset({
        features: ['gendered', 'neutral'],
        num_rows: 96
    })
})

In [6]:
dataset['train']

Dataset({
    features: ['gendered', 'neutral'],
    num_rows: 3513
})

In [7]:
dataset['train'].to_pandas()

,gendered,neutral
0,Se han incorporado al centro dos docentes para...,Se han incorporado al centro dos docentes para...
1,Tenemos que cubrir las carencias que presenta ...,Tenemos que cubrir las carencias que presenta ...
2,Habla con tu médico para averiguar si necesit...,Habla con tu médico para averiguar si necesit...
3,"Mientras tanto , el espectador comenzaba a im...","Mientras tanto , la audiencia comenzaba a imp..."
4,Los intermediarios del acuerdo se ocuparán tam...,El equipo intermediadior del acuerdo se ocupar...
...,...,...
3508,Es importante cubrir las carencias que tiene e...,Es importante cubrir las carencias que tiene e...
3509,la empleada doméstica ha terminado su trabajo ...,el servicio doméstico ha terminado su trabajo ...
3510,Las páginas web podrán publicar haciendo const...,Las páginas web podrán publicar haciendo const...
3511,realizamos entrevistas con los encargados de ...,realizamos entrevistas con el personal respon...


In [8]:
from transformers import AutoTokenizer

model_checkpoint = "facebook/mbart-large-50"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

tokenizer_config.json:   0%|          | 0.00/531 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/649 [00:00<?, ?B/s]

In [9]:
max_length = 128


def preprocess_function(examples):
    model_inputs = tokenizer(
        examples["gendered"],
        max_length=max_length,
        truncation=True,
    )
    labels = tokenizer(
        examples["neutral"], max_length=max_length, truncation=True
    )
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [10]:
tokenized_datasets = dataset.map(preprocess_function, batched=True)
tokenized_datasets

Map:   0%|          | 0/3513 [00:00<?, ? examples/s]

Map:   0%|          | 0/96 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['gendered', 'neutral', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 3513
    })
    test: Dataset({
        features: ['gendered', 'neutral', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 96
    })
})

In [11]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer)

# MODELO

In [12]:
from transformers import Seq2SeqTrainingArguments


args = Seq2SeqTrainingArguments(
    # Carpeta donde se guardará el modelo
    output_dir="mbart-neutralization",
    # Evaluar el modelo cada época
    evaluation_strategy="epoch",
    # Valores para modificar los pesos del modelo
    learning_rate=5.6e-5,
    weight_decay=0.01,
    # Número de ejemplos que se muestran cada vez al modelo
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    # Número de veces que que se muestran todos los datos al modelo
    num_train_epochs=2,
    # Realizar predicciones al validar
    predict_with_generate=True,
    push_to_hub=True,
    report_to="none"
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [13]:
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

pytorch_model.bin:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/261 [00:00<?, ?B/s]

In [14]:
import numpy as np

import evaluate

metric = evaluate.load("sacrebleu")

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

In [15]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

<ipython-input-15-7e05dcb5bafc>:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [16]:
trainer.train()

Epoch,Training Loss,Validation Loss,Bleu,Gen Len
1,No log,0.017277,96.418200,26.864600
2,0.224800,0.016453,96.237900,18.781200


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:2810: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 200, 'early_stopping': True, 'num_beams': 5}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


TrainOutput(global_step=880, training_loss=0.13868910724466496, metrics={'train_runtime': 936.0863, 'train_samples_per_second': 7.506, 'train_steps_per_second': 0.94, 'total_flos': 519539733135360.0, 'train_loss': 0.13868910724466496, 'epoch': 2.0})

# COMPARTIENDO Y USANDO EL MODELO

In [17]:
%cd mbart-neutralization
trainer.push_to_hub(commit_message="Training complete", tags="simplification")

/content/mbart-neutralization


CommitInfo(commit_url='https://huggingface.co/manapole/mbart-neutralization/commit/ecb8d520893e1f60c4b20cb56769660493f39029', commit_message='Training complete', commit_description='', oid='ecb8d520893e1f60c4b20cb56769660493f39029', pr_url=None, repo_url=RepoUrl('https://huggingface.co/manapole/mbart-neutralization', endpoint='https://huggingface.co', repo_type='model', repo_id='manapole/mbart-neutralization'), pr_revision=None, pr_num=None)

In [18]:
from transformers import pipeline
neutralizer = pipeline('text2text-generation', model='manapole/mbart-neutralization')

config.json:   0%|          | 0.00/1.41k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/226 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/10.9k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/992 [00:00<?, ?B/s]

Device set to use cuda:0


In [19]:
neutralizer('El profesor de la asignatura decidirá cómo hacer el examen')

[{'generated_text': 'El profesor de la asignatura decidirá cómo hacer el examen'}]

In [20]:
neutralizer('Mañana habrá un partido de fútbol')

[{'generated_text': 'Mañana habrá un partido de fútbol'}]

In [21]:
neutralizer('Hoy comeremos en casa de la abuela')

[{'generated_text': 'Hoy comeremos en casa de la abuela'}]

## Ejercicios adicionales

1. Utiliza un dataset de traducción distinto al de neutralización (2.5 puntos).
2. Utiliza un modelo distinto al de mbart (puedes usar modelos como t5 o pegasus) (2.5 puntos).

Si haces estas tareas debes crear un nuevo notebook de Colaboratory (una para cada una de ellas) y guardarlo en tu repositorio de GitHub.